Cargamos los archivos depurados

In [96]:
import pandas as pd

In [97]:
#Creamos una funcion para mostrar los tipos de columnas, cantidad de filas nulas y % de nulos
def resumen(df):
    print(f'data shape: {df.shape}')
    resumen = pd.DataFrame(df.dtypes, columns=['data type'])
    resumen['#missing'] = df.isnull().sum()
    resumen['%missing'] = df.isnull().sum()*100/len(df)
    resumen['#unique'] = df.nunique()
    return resumen

In [40]:
#Cargamos la base Steam Games, pero descartamos las columnas que no usaremos

# Especificamos las columnas que deseamos cargar
columnas_deseadas = ['title',	'id',	'Action',	'Casual',	'Indie',	'Simulation',	'Strategy',	'Free to Play',	'RPG',	'Sports',	'Adventure',	'Racing',	'Early Access',	'Design & Illustration',	'Massively Multiplayer',	'Education',	'Animation & Modeling',	'Video Production',	'Utilities',	'Web Publishing',	'Software Training',	'Audio Production',	'Photo Editing',	'Accounting',	'anho_lanzamiento']

# Cargamos el DataFrame desde el archivo parquet con solo las columnas deseadas
df_games = pd.read_parquet('df_steam_games.parquet' , columns=columnas_deseadas)
df_games.head()



,title,id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,...,Education,Animation & Modeling,Video Production,Utilities,Web Publishing,Software Training,Audio Production,Photo Editing,Accounting,anho_lanzamiento
88310,Lost Summoner Kitty,761140.0,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,2018
88311,Ironbound,643980.0,False,False,True,False,True,True,True,False,...,False,False,False,False,False,False,False,False,False,2018
88312,Real Pool 3D - Poolians,670290.0,False,True,True,True,False,True,False,True,...,False,False,False,False,False,False,False,False,False,2017
88313,弹炸人2222,767400.0,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2017
88314,Log Challenge,773570.0,True,True,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,<NA>


In [98]:
#Cargamos las bases User Items y User Reviews,

df_items = pd.read_parquet('df_items.parquet')
df_reviews = pd.read_parquet('df_reviews.parquet')


df_items.head()


,user_id,item_id,playtime_forever
0,76561197970982479,10,6
2,76561197970982479,30,7
8,76561197970982479,300,4733
9,76561197970982479,240,1853
10,76561197970982479,3830,333


In [45]:
df_reviews.head()

,user_id,item_id,recommend,sentiment_analysis,anho_posteo
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011
2,76561197970982479,43110,True,2,2011
3,js41637,251610,True,2,2014
4,js41637,227300,True,0,2013


In [41]:
# Con Steam Games, para mejor analisis, trasponemos los generos como filas

# Lista de columnas que son dummies de Genero
columnas_genero = ['Action',	'Casual',	'Indie',	'Simulation',	'Strategy',	'Free to Play',	'RPG',	'Sports',	'Adventure',	'Racing',	'Early Access',	'Design & Illustration',	'Massively Multiplayer',	'Education',	'Animation & Modeling',	'Video Production',	'Utilities',	'Web Publishing',	'Software Training',	'Audio Production',	'Photo Editing',	'Accounting']

# Inicializamos una lista vacia
nuevas_filas = []

# Iteramos sobre cada fila del DataFrame 
for index, fila in df_games.iterrows():
    item_id = fila['id']
    title = fila['title']
    anho_lanzamiento = fila['anho_lanzamiento']
    
    # Filtramos las columnas dummies que tienen el valor verdadero
    generos = [columna for columna in columnas_genero if fila[columna] == True]
    
    # Creamos filas adicionales para cada genero
    for genero in generos:
        nuevas_filas.append({'title': title,'item_id': item_id,'anho_lanzamiento': anho_lanzamiento, 'Genero': genero})

df_games_vf = pd.DataFrame(nuevas_filas)


In [42]:
df_games_vf.tail()

,title,item_id,anho_lanzamiento,Genero
77982,Maze Run VR,681550.0,<NA>,Action
77983,Maze Run VR,681550.0,<NA>,Indie
77984,Maze Run VR,681550.0,<NA>,Simulation
77985,Maze Run VR,681550.0,<NA>,Adventure
77986,Maze Run VR,681550.0,<NA>,Early Access


In [43]:
df_games_vf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77987 entries, 0 to 77986
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             77987 non-null  object 
 1   item_id           77987 non-null  float64
 2   anho_lanzamiento  71100 non-null  object 
 3   Genero            77987 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.4+ MB


Programamos la primera funcion.

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [83]:
resumen(df_games_vf)
#Tenemos items duplicados por que los repetimos por cada genero que tiene

data shape: (77987, 4)


,data type,#missing,%missing,#unique
title,object,0,0.000000,31175
item_id,float64,0,0.000000,31206
anho_lanzamiento,object,6887,8.830959,39
Genero,object,0,0.000000,22


In [84]:
print(df_games_vf.sort_values(by='item_id', ascending=False))

                                         title    item_id anho_lanzamiento  \
2427      Bioshock Infinite: Columbia's Finest  2028850.0             2013   
75149        Assassin’s Creed® III Season Pass  2028103.0             2012   
75148        Assassin’s Creed® III Season Pass  2028103.0             2012   
74784  Call of Duty®: Black Ops II Season Pass  2028062.0             <NA>   
75467             Worms Revolution Season Pass  2028056.0             2012   
...                                        ...        ...              ...   
77924                Half-Life: Opposing Force       50.0             1999   
77929                       Deathmatch Classic       40.0             2001   
77940                            Day of Defeat       30.0             2003   
77923                    Team Fortress Classic       20.0             1999   
77926                           Counter-Strike       10.0             2000   

          Genero  
2427      Action  
75149  Adventure  
75148 

In [74]:
# Filtramos filas con 'anho_lanzamiento' no vacíos en df_games_vf
df_games_vf_filtrado = df_games_vf[df_games_vf['anho_lanzamiento'].notnull()]
resumen(df_games_vf_filtrado)

data shape: (71100, 4)


,data type,#missing,%missing
title,object,0,0.0
item_id,float64,0,0.0
anho_lanzamiento,object,0,0.0
Genero,object,0,0.0


In [99]:
resumen(df_items)

data shape: (3246362, 3)


,data type,#missing,%missing,#unique
user_id,object,0,0.0,68403
item_id,Int64,0,0.0,10050
playtime_forever,int64,0,0.0,48860


In [100]:
# Antes de cruzar, sumarizamos el 'playtime_forever' de todos los usuarios agrupados por 'item_id'
df_horas_x_item = df_items.groupby('item_id')['playtime_forever'].sum().reset_index()
print(df_horas_x_item.shape)
df_horas_x_item.head()

(10050, 2)


,item_id,playtime_forever
0,10,17107858
1,20,960524
2,30,756375
3,40,154424
4,50,726545


In [106]:
# Realizamos el cruce de games con items por la columna 'item_id' después de aplicar los filtros
games_and_items = pd.merge(df_games_vf_filtrado, df_horas_x_item, left_on=['item_id'], right_on=['item_id'], how='inner')
resumen(games_and_items)
#Al hacer un left join verificamos que hay juegos en la base Items que no cruzaron con la base Games...por lo que para esta funcion necesitamos hacer un inner necesariamente
#para tener el dato de genero y anho de lanzamiento

data shape: (20106, 5)


,data type,#missing,%missing,#unique
title,object,0,0.0,8120
item_id,float64,0,0.0,8123
anho_lanzamiento,object,0,0.0,34
Genero,object,0,0.0,21
playtime_forever,int64,0,0.0,6250


In [115]:
#Agrupamos los datos por genero y anho de lanzamiento, sumarizando playtime_forever
df_PlayTimeGenre = games_and_items.groupby(['Genero', 'anho_lanzamiento'])['playtime_forever'].sum().reset_index()

In [116]:
#Ordenamos los datos por genero segun el tiempo jugado mayor 
df_PlayTimeGenre = df_PlayTimeGenre.sort_values(by=['Genero', 'playtime_forever'], ascending=[True, False])

In [117]:
df_PlayTimeGenre

,Genero,anho_lanzamiento,playtime_forever
26,Action,2012,1085700047
27,Action,2013,415522341
25,Action,2011,331795530
29,Action,2015,251646844
24,Action,2010,180087364
...,...,...,...
332,Web Publishing,2005,93844
336,Web Publishing,2014,33641
333,Web Publishing,2007,11058
339,Web Publishing,2017,9382


In [118]:
def PlayTimeGenre(genero):
    # Filtrar por género
    df_genero = df_PlayTimeGenre[df_PlayTimeGenre['Genero'] == genero]
    
    # Agrupar por año y sumarizar las horas jugadas
    df_agrupado = df_genero.groupby('anho_lanzamiento')['playtime_forever'].sum().reset_index()
    
    # Encontrar el año con más horas jugadas
    anho_max_horas = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax(), 'anho_lanzamiento']
    
    return {"Año de lanzamiento con más horas jugadas para Género {}: ".format(genero): anho_max_horas}


In [121]:
resultado = PlayTimeGenre('Strategy')
print(resultado)

{'Año de lanzamiento con más horas jugadas para Género Strategy: ': 2013}
